# TEXT MINING for PRACTICE
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : 전병진(fingeredman@gmail.com)**

---

## WEEK 11-1. 뉴스기사를 활용한 KOSPI 등락여부 예측하기
- Python으로 비정형 데이터를 활용해 KOSPI 등락여부를 예측하는 방법에 대해 다룹니다.

---

In [1]:
from teanaps import configure as con
PLOTLY_USERNAME = con.PLOTLY_USERNAME
PLOTLY_API_KEY = con.PLOTLY_API_KEY

import plotly 
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly import tools
plotly.tools.set_credentials_file(username=PLOTLY_USERNAME, api_key=PLOTLY_API_KEY)
plotly.tools.set_config_file(world_readable=False, sharing='private')
init_notebook_mode(connected=True)

from teanaps.nlp.SyntaxAnalyzer import SyntaxAnalyzer
from teanaps.text_analysis.TfidfCalculator import TfidfCalculator
from teanaps.text_analysis.TopicClustering import TopicClustering

#import statsmodels.api as sm
#from statsmodels.tsa.api import VAR, DynamicVAR
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
import numpy as np
import glob

In [2]:
df = pd.read_csv('data/kospi.txt', sep="\t") 
df["y"] = [float(y.replace(",", "")) for y in df["y"]]
df

,ds,y,오픈,고가,저가,거래량,변동폭
0,2019년 5월,2061.25,"2,192.94","2,217.03","2,051.69",-,-6.46%
1,2019년 4월,2203.59,"2,153.31","2,252.05","2,153.31",9.69B,2.94%
2,2019년 3월,2140.67,"2,210.97","2,216.02","2,120.93",6.46B,-2.49%
3,2019년 2월,2195.44,"2,211.93","2,241.76","2,167.36",6.88B,-0.43%
4,2019년 1월,2204.85,"2,050.55","2,222.88","1,984.53",9.46B,8.03%
5,2018년 12월,2041.04,"2,127.78","2,136.64","2,014.28",7.63B,-2.66%
6,2018년 11월,2096.86,"2,035.11","2,136.74","2,023.43",6.98B,3.31%
7,2018년 10월,2029.69,"2,349.64","2,352.11","1,985.95",7.02B,-13.37%
8,2018년 9월,2343.07,"2,317.93","2,356.62","2,264.90",5.43B,0.87%
9,2018년 8월,2322.88,"2,301.17","2,322.88","2,218.09",6.20B,1.20%


In [3]:
sa = SyntaxAnalyzer()

document_list = []
tokenized_sentence_list = []

POS_LIST = ["NNG", "NNP"]

num_monthly_documents = 100

file_list = glob.glob("data/monthly_data/*")
file_list.sort()
for filename in file_list:
    count = 1
    line_list = open(filename, encoding="utf-8").read().split("\n")
    for line in line_list:
        if count > num_monthly_documents:
            continue
        if len(line) > 100:
            count += 1
            col = line.split("\t")
            datetime = col[0]
            month = datetime[:7]
            content = col[1]
            document_list.append(content)
            tagged_word_list = sa.parse(content)
            tokenized_sentence = ""
            for word, pos in tagged_word_list:
                if pos in POS_LIST:
                    tokenized_sentence += " " + word
            tokenized_sentence_list.append(tokenized_sentence.strip())

In [4]:
tokenized_sentence_list[:5]

['귀주 가스 석탄 사용 금지 가격 급등 관련 상장 주식 대부분 연상 경향',
 '안녕 중국 거주 얼마 전 중국 주식 시장 투자 회사 투자 생각 글 귀주 가스 얼마 후 매입 갑자 장 정지 일전 프로 수익 매도 이후 한지 얼마 화 능수 전 투자 한지 얼 회사 투자 중국 주식 모두 성',
 '언어 매너 활동 알찬 해복 주위 가상 화폐 투자 기본 이득 지인 전 윌 투자 결과 현재 속 뉴스 한 코스닥 상승 열풍 주식 시장 폐장 코스닥 상장 시가총액 실체 리플 직 더리 움 가상 화폐 시가 총액 대한민국 미래 기업 전체 시가총액 가치 고수 고견 한 행복 정보 공유 비트 무료 리딩 매너 회원 신고 투자 본인 책임 정보 참고 판단 매너 반말 비속어 공격 어휘 방 홍보 시 경고 영구 강퇴 비트코인 비트 동호회 비트코인 채굴 라이트 코인 리플 가상 화폐 투자 정보 비트코인 시세 퀀텀 오스 비트코인 전망 비트 캐쉬 캐시 대쉬 클래식 빗 썸 업 비트 코 인원 코인 네스트',
 '존 버 기회 개인 욕심 급등 코인 손절 반복 결국 욕심 해 성 투하 쇼',
 '투자 코인 방법 걱정 스타일 폰']

In [5]:
tfidf = TfidfCalculator()

df_tfidf_monthly = []
for i in range(11):
    tokenized_sentence_list_monthly = tokenized_sentence_list[i*100:(i+1)*100]
    #tokenized_sentence_list_monthly
    tfidf.calculation_tfidf(tokenized_sentence_list_monthly)
    df_tfidf = tfidf.get_tfidf_matrix()
    #df_tfidf
    df_tfidf_monthly.append(df_tfidf)

In [6]:
df_tfidf_monthly[10][:20]

com  company  core  coredictionary  dictionary  fod  how  koreans  market  \
0   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
1   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
2   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
3   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
4   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
5   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
6   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
7   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
8   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
9   0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
10  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
11  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
12  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
13  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
14  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
15  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
16  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
17  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
18  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   
19  0.0      0.0   0.0             0.0         0.0  0.0  0.0      0.0     0.0   

    rice  share  staple  stock  today  yons        가감        가격  가격대  가격표  \
0    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
1    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.179378  0.0  0.0   
2    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.049820  0.0  0.0   
3    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.034928  0.0  0.0   
4    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
5    0.0    0.0     0.0    0.0    0.0   0.0  0.056912  0.000000  0.0  0.0   
6    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
7    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
8    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
9    0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
10   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
11   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
12   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
13   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
14   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
15   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
16   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
17   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
18   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   
19   0.0    0.0     0.0    0.0    0.0   0.0  0.000000  0.000000  0.0  0.0   

    가급적        가능   가담   가량       가르침   가망        가분        가상   가슴        가시  \
0   0.0  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.122317  0.0  0.000000   
1   0.0  0.012563  0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000   
2   0.0  0.000000  0.0  0.0  0.073771  0.0  0.000000  0.000000  0.0  0.000000   
3   0.0  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.193701  0.0  0.000000   
4   0.0  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000   
5   0.0  0.000000  0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000   
6   0.0  0.000000  0.0  0.0  0.000000  0.0  0.062031  0.000000  0.0  0.000000   
7   0.0  0.000000  0.0  0.0  0.00000

In [7]:
tc = TopicClustering()

NUM_TOPICS = 10
NUM_KEYWORDS = 10

lda_result = tc.lda_topic_modeling(tokenized_sentence_list, NUM_TOPICS, NUM_KEYWORDS)
lda_result

[(0,
  [('글', 0.021350402),
   ('생각', 0.018462282),
   ('주식', 0.017549278),
   ('강의', 0.012964951),
   ('종목', 0.012241725),
   ('수익', 0.010073846),
   ('사람', 0.009764216),
   ('시장', 0.009671222),
   ('투자', 0.008944308),
   ('감사', 0.008742424)]),
 (1,
  [('아파트', 0.014912637),
   ('울산', 0.010381873),
   ('가격', 0.009981945),
   ('혁신', 0.009908048),
   ('도시', 0.00974713),
   ('지역', 0.007990665),
   ('기관', 0.0075416476),
   ('직원', 0.007061061),
   ('부동산', 0.0068981266),
   ('때', 0.006069463)]),
 (2,
  [('시장', 0.022956641),
   ('주식', 0.018506672),
   ('투자', 0.009153264),
   ('수익', 0.008438933),
   ('가격', 0.00794772),
   ('말', 0.0067507476),
   ('부동산', 0.006443193),
   ('화폐', 0.0064308005),
   ('생각', 0.006332065),
   ('나라', 0.0061554494)]),
 (3,
  [('금리', 0.020543056),
   ('시장', 0.020084888),
   ('미국', 0.013040213),
   ('투자', 0.012000502),
   ('금융', 0.010865243),
   ('주식', 0.010812674),
   ('상승', 0.00963194),
   ('중국', 0.008443808),
   ('경제', 0.0073373476),
   ('하락', 0.006867659)]),
 (4,
  [(

In [8]:
hot_keyword_list = []

for num_topics, keyword_list in lda_result:
    for keyword, score in keyword_list:
        if keyword not in hot_keyword_list:
            hot_keyword_list.append(keyword)
            
hot_keyword_dict = {}

for hot_keyword in hot_keyword_list:
    tfidf_series = []
    for df_tfidf in df_tfidf_monthly:
        tfidf_dict = dict(df_tfidf.sum())
        if hot_keyword in tfidf_dict.keys():
            tfidf_series.append(tfidf_dict[hot_keyword])
        else:
            tfidf_series.append(0)
    tfidf_series2 = []
    for idx in range(len(tfidf_series)-1):
        if tfidf_series[idx] > tfidf_series[idx+1]:
            tfidf_series2.append(0)
        else:
            tfidf_series2.append(1)
    hot_keyword_dict[hot_keyword] = tfidf_series2

In [9]:
d = df[["y"]].values[6:17]
d = list(d)
d = [float(dd) for dd in d]
d

[2096.86,
 2029.69,
 2343.07,
 2322.88,
 2295.26,
 2326.13,
 2423.01,
 2515.38,
 2445.85,
 2427.36,
 2566.46]

In [10]:
x = [i for i in range(11)]

data = [go.Scatter(
        x=x,
        y=d,
        name="KOSPI",
        marker=dict(
            #color='blue',
        ),
        yaxis='y2'
    )]


for key, value in hot_keyword_dict.items():
    y = value
    data.append(go.Scatter(
        x=x,
        y=y,
        name=key,
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    ),)
layout = go.Layout(
    title='Sequence LDA Model Topic Trend',
    xaxis=dict(
        title='TIME SLICE',
        titlefont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='WEIGHT',
        titlefont=dict(
            size=10,
            color='black'
        ),
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all',
        #overlaying='y',
    ),
    yaxis2=dict(
                title='TF-IDF',
                titlefont=dict(
                    size=10,
                    color='black'
                ),
                showticklabels=True,
                tickangle=0,
                tickfont=dict(
                    size=10,
                    color='black'
                ),
                overlaying='y',
                side='right'
            ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='TF-IDF Graph')

In [11]:
y = df["y"].values[6:17]
ud_y = []

for y_idx in range(len(y)-1):
    print(y[y_idx], y[y_idx+1])
    if y[y_idx] < y[y_idx+1]:
        ud_y.append(1)
    else:
        ud_y.append(0)

2096.86 2029.69
2029.69 2343.07
2343.07 2322.88
2322.88 2295.26
2295.26 2326.13
2326.13 2423.01
2423.01 2515.38
2515.38 2445.85
2445.85 2427.36
2427.36 2566.46


In [12]:
Y = pd.DataFrame(ud_y)
Y = Y[0].values

X = pd.DataFrame(hot_keyword_dict)
X = X.values[:]

df = pd.DataFrame(X, columns = hot_keyword_dict.keys())
df['target'] = Y
df

,글,생각,주식,강의,종목,수익,사람,시장,투자,감사,아파트,울산,가격,혁신,도시,지역,기관,직원,부동산,때,말,화폐,나라,금리,미국,금융,상승,중국,경제,하락,뉴스,보통주,무역,美,연합뉴스,트럼프,미,매수,펀드,매도,코인,돈,한국,기업,주가,정보,주,거래,비상,장주식,삼성,장외,바이오,일본,증권,은행,대비,실적,대출,target
0,1,0,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,0,1,1,1,0,0,0,0,0,1,0,0,1,1,1,0,1,1,1,1,1,0,0,1,1,0,0,1,1,1,0,0,1,1,1,1,0,0,1,0,0,0,0
1,1,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,1,1,0,0,0,0,1
2,1,0,1,0,1,1,0,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
3,1,1,0,0,0,1,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0
4,1,1,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,1
5,1,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,0,1
6,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,1,1,0,1,1,1,1,1,1,0,0,0,1,1,1,0,0,1,1
7,1,1,0,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0,1,0,0,1,0,0
8,1,1,0,1,1,1,1,0,1,1,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,1,1,0,0,1,1,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0
9,1,1,1,0,0,1,0,1,0,0,0,1,1,1,0,0,0,1,0,1,1,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,1,1,1,0,1,1,0,1,0,0,1,0,0,0,1,0,0,1,1,1,1,1,1,1


In [13]:
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=None)
    dtc = DecisionTreeClassifier(random_state=0)
    #dtc = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
    dtc.fit(x_train, y_train)
    score = dtc.score(x_test, y_test)
    print('Accuracy', i+1, ' :', score)

Accuracy 1  : 1.0
Accuracy 2  : 1.0
Accuracy 3  : 1.0
Accuracy 4  : 1.0
Accuracy 5  : 1.0
Accuracy 6  : 1.0
Accuracy 7  : 1.0
Accuracy 8  : 1.0
Accuracy 9  : 1.0
Accuracy 10  : 1.0


In [14]:
x = [i for i in range(11)]

data = [go.Scatter(
        x=x,
        y=d,
        name="KOSPI",
        marker=dict(
            #color='blue',
        ),
        yaxis='y2'
    ),
    go.Scatter(
        x=x,
        y=ud_y,
        name="Label",
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    ),
    go.Scatter(
        x=x,
        y=dtc.predict(X),
        name="Predict",
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    )]

layout = go.Layout(
    title='Predictive Analysis',
    xaxis=dict(
        title='TIME SLICE',
        titlefont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='DOWN <-------------------------------------------> UP',
        titlefont=dict(
            size=10,
            color='black'
        ),
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        exponentformat='e',
        showexponent='all',
        #overlaying='y',
    ),
    yaxis2=dict(
                title='KOSPI200',
                titlefont=dict(
                    size=10,
                    color='black'
                ),
                showticklabels=True,
                tickangle=0,
                tickfont=dict(
                    size=10,
                    color='black'
                ),
                overlaying='y',
                side='right'
            ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='TF-IDF Graph')